In [1]:
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

In [2]:
traindf = pd.read_csv('data/fixed_train.csv')
validdf = pd.read_csv('data/fixed_valid.csv')
testdf = pd.read_csv('data/fixed_test.csv')

In [3]:
# replace "_comma_" with ","
def replace_comma(df):
    df.loc[:, 'prompt'] = df['prompt'].str.replace('_comma_', ',')
    df.loc[:, 'utterance'] = df['utterance'].str.replace('_comma_', ',')

replace_comma(traindf)
replace_comma(validdf)
replace_comma(testdf)

In [4]:
def concate_conv(df):
    df_concat = pd.DataFrame(columns=df.columns) \
                .drop(['utterance_idx'], axis=1) \
                .rename(columns={'utterance': 'conv'})

    conv_id_groups = df.groupby(['conv_id'])

    for _, indices in tqdm(conv_id_groups.groups.items()):
        conv_rows = df.loc[indices].copy().sort_values(['utterance_idx'])
        conv_row = conv_rows.iloc[0].drop(['utterance_idx']).rename({'utterance': 'conv'})
        concat_text = \
            ' '.join([(row['utterance'] ) for _, row in conv_rows.iterrows()])
        conv_row['conv'] = concat_text
        df_concat = df_concat.append(conv_row, ignore_index=True)
        
    return df_concat

_traindf = concate_conv(traindf)
_validdf = concate_conv(validdf)
_testdf = concate_conv(testdf)

In [5]:
endl = '\n'
print(f'train:{endl}{endl.join(_traindf["conv"].values[:2].tolist())}{endl}')
print(f'valid:{endl}{endl.join(_validdf["conv"].values[:2].tolist())}{endl}')
print(f'test:{endl}{endl.join(_testdf["conv"].values[:2].tolist())}{endl}')

train:
I remember going to see the fireworks with my best friend. It was the first time we ever spent time alone together. Although there was a lot of people, we felt like the only people in the world. Was this a friend you were in love with, or just a best friend? This was a best friend. I miss her. Where has she gone? We no longer talk. Oh was this something that happened because of an argument?
My girlfriend got me a pet toad today! Do you like toads? I do! I was so happy when I opened the box and that fat mofo jumped out! That was nice of your girlfriend, do you love her?

valid:
My upstairs neighbors make a ton of noise at all hours of the night. It makes it difficult for me to sleep.  That really sucks. Maybe you should try egging their door? Or just break in and pretend you're bigfoot while they're trying to sleep. I'm not trying to get arrested! I think I'll just wait things out until I move in two months. I would go with the bigfoot option. You can get a costume on the cheap o

In [6]:
_traindf.to_csv('data/new_train_0608.csv', index=False, encoding='utf8')
_validdf.to_csv('data/new_valid_0608.csv', index=False, encoding='utf8')
_testdf.to_csv('data/new_test_0608.csv', index=False, encoding='utf8')